In [ ]:
# Define a Function to Fetch Metadat
def get_movie_metadata(title, api_key):
    # Search for movie ID
    search_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}"
    search_response = requests.get(search_url).json()
    
    if not search_response["results"]:
        return None
    
    movie_id = search_response["results"][0]["id"]
    
    # Get credits (cast and crew)
    credits_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}"
    credits_response = requests.get(credits_url).json()
    
    actors = [member["name"] for member in credits_response.get("cast", [])[:5]]
    directors = [member["name"] for member in credits_response.get("crew", []) if member["job"] == "Director"]
    
    return {"actors": actors, "directors": directors}

In [ ]:
# Apply to Sample Titles
sample_titles = {
    0: "The Dark Knight",
    1: "Titanic",
    2: "Inception"
}

metadata_map = {}

for idx, title in sample_titles.items():
    metadata = get_movie_metadata(title, API_KEY)
    metadata_map[idx] = metadata

# Attach metadata to reviews
for idx, meta in metadata_map.items():
    train_df.loc[idx, "actors"] = ", ".join(meta["actors"]) if meta else None
    train_df.loc[idx, "directors"] = ", ".join(meta["directors"]) if meta else None

# Preview enriched DataFrame
print(train_df.loc[list(sample_titles.keys()), ["text", "actors", "directors"]])

In [ ]:
#Link Reviews to Metadata
import pandas as pd
import requests

# Load IMDb dataset
from datasets import load_dataset
dataset = load_dataset("stanfordnlp/imdb")
train_df = pd.DataFrame(dataset["train"])

# Your TMDb API key
API_KEY = "your_tmdb_api_key"  # Replace with your actual key

In [ ]:
#Define Metadata Fetch Function
def get_tmdb_metadata(title, api_key):
    search_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}"
    search_response = requests.get(search_url).json()
    
    if not search_response["results"]:
        return None
    
    movie_id = search_response["results"][0]["id"]
    
    credits_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}"
    credits_response = requests.get(credits_url).json()
    
    actors = [member["name"] for member in credits_response.get("cast", [])[:5]]
    directors = [member["name"] for member in credits_response.get("crew", []) if member["job"] == "Director"]
    
    return {"actors": actors, "directors": directors}

In [ ]:
#Manually Map Sample Reviews to Movie Titles
sample_titles = {
    0: "The Dark Knight",
    1: "Titanic",
    2: "Inception"
}

In [ ]:
#Fetch Metadata and Attach to DataFrame
metadata_map = {}

for idx, title in sample_titles.items():
    metadata = get_tmdb_metadata(title, API_KEY)
    metadata_map[idx] = metadata

# Add metadata to DataFrame
for idx, meta in metadata_map.items():
    if meta:
        train_df.loc[idx, "actors"] = ", ".join(meta["actors"])
        train_df.loc[idx, "directors"] = ", ".join(meta["directors"])
    else:
        train_df.loc[idx, "actors"] = None
        train_df.loc[idx, "directors"] = None

# Preview enriched rows
print(train_df.loc[list(sample_titles.keys()), ["text", "actors", "directors"]])